In [1]:
# import the necessary packages
from imutils.video import FileVideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import math
import time
import dlib
import cv2
import numpy as np
import sys
import subprocess as sp
from sklearn.decomposition import PCA
from PIL import Image
import csv
from os import listdir
from scipy.spatial.distance import pdist, squareform
import weightedstats as ws
import shutil

w = []

In [2]:
# load landmark detector
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = face_utils.FaceAligner(predictor, desiredFaceWidth=256)

[INFO] loading facial landmark predictor...


In [3]:
# for testing purpose
def lip_detect(frame):
        # load the input image, resize it, and convert it to grayscale
        clone = np.zeros([256,256,3],dtype=np.uint8)
        image = cv2.imread(frame)
        image = imutils.resize(image, width=256)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)

        # loop over the face detections
        for (i, rect) in enumerate(rects):
            # determine the facial landmarks for the face region, then
            # convert the landmark (x, y)-coordinates to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # loop over the face parts individually
            for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
                # clone the original image so we can draw on it, then
                # display the name of the face part on the image
                if name == "mouth":
                    # loop over the subset of facial landmarks, drawing the
                    # specific face part
                    for (x, y) in shape[i:j]:
                        cv2.circle(clone, (x, y), 2, (0, 0, 255), -1)
                        cv2.circle(image, (x, y), 2, (0, 0, 255), -1)

        return clone, image

In [4]:
# function to calculate the weighted median 
def weighted_median(data, midpoint, weights=None):
    import numpy as np
    if weights is None:
        return np.median(np.array(data).flatten())
    data, weights = np.array(data).flatten(), np.array(weights).flatten()
    if any(weights > 0):
        sorted_data, sorted_weights = map(np.array, zip(*sorted(zip(data, weights))))
        if any(weights > midpoint):
            return (data[weights == np.max(weights)])[0]
        cumulative_weight = np.cumsum(sorted_weights)
        below_midpoint_index = np.where(cumulative_weight <= midpoint)[0][-1]
        if cumulative_weight[below_midpoint_index] - midpoint < sys.float_info.epsilon:
            return np.mean(sorted_data[below_midpoint_index:below_midpoint_index+2])
    return sorted_data[below_midpoint_index+1]

In [5]:
# don't use
# function to calculate sigma (to be changed)
def calcSigma(S_n, S_N, S_target, n, N):
    sigma = 0
    temp1 = 0
    t = 0
    for i in range(1, 1000):
        # print("for i="+str(i/10))
        x2 = np.sum(calcWeight(S_N, S_target, N, i/10))
        x1 = np.sum(calcWeight(S_n, S_target, n, i/10))
        temp = x1 - (0.9*x2)
        print(temp)
        if t == 0:
            sigma = i/10
            temp1 = math.fabs(temp)
            t = 1
        else:
            if math.fabs(temp) < temp1:
                sigma = i/10
                temp1 = math.fabs(temp)
    # print('sigma is'+str(sigma))
    return sigma

In [6]:
# function to calculate sigma (to be changed)
# to be used
def calcSigma(S_n, S_N, S_target, n, N):
    left = 1
    right = 1000
    while left < right:
        mid = int(left + (right - left) / 2)
        start = np.sum(calcWeight1(S_n, S_target, n, left/10)) - 0.9*np.sum(calcWeight1(S_N, S_target, N, left/10))
        end = np.sum(calcWeight1(S_n, S_target, n, right/10)) - 0.9*np.sum(calcWeight1(S_N, S_target, N, right/10))
#         print('left '+str(start))
#         print('right '+str(end))
        if start > end:
            right = mid
        else:
            left = mid+1
    return left

In [6]:
# function to calculate weight, given sigma (to be changed to include sigma)
# legacy
def calcWeight(S, S_target, len, sigma):
    sqSum=0
    # print(S)
#     # print(S_target)
#     S = S.astype(int)
#     S_target = S.astype(int)
    global w 
    w = []
    for i in range(len):
        sqSum = 0.0
        for j in range(25):
            sqSum += (S[i, j]-S_target[j])**2
            p = sqSum/(2*sigma*sigma)
        w = np.append(w, np.exp(-p))
    print(w)
    return w

In [7]:
# function to calculate weight, given sigma (to be changed to include sigma)
# legacy
def calcWeight(S, S_target, len, sigma):
    sqSum=0
    # print(S)
    # print(S_target)
    global w 
    w = []
    for i in range(len):
        sqSum = 0
        for j in range(36):
            sqSum += (S[i, j]-S_target[j])^2
            p = sqSum/(2*sigma*sigma)
        w = np.append(w, np.exp(-p))
    # print(w)
    return w

In [7]:
# function to calculate weight, given sigma (to be changed to include sigma)
# to be used
def calcWeight(S, S_target, len, sigma):
    # print(S)
    # print(S_target)
    const = 2*(sigma**2)
    global w 
    w = []
    for i in range(len):
        z = (np.linalg.norm(S[i] - S_target))**2
        w = np.append(w, np.exp(-z/const))
        # print(np.linalg.norm(S[i] - S_target))
    # print(w)
    return w

In [8]:
# function to calculate weight, given sigma (to be changed to include sigma)
# to be used
def calcWeight1(S, S_target, len, sigma):
    # print(S)
    # print(S_target)
    const = 2*(sigma**2)
    global w 
    w = []
    for i in range(len):
        z = (np.linalg.norm(S[i] - S_target))**2
        w = np.append(w, np.exp(-z/const))
        # print(z)
    # print(w)
    return w

In [6]:
# don't use
def weigtedMedianPP(img, frame, x, y, n, w, sum_w):
    val = cv2.imread(frame)[x][y]
    median_pixel = np.zeros(3)
    weighted_sum = np.zeros(3)
    for i in range(n):
        c = cv2.imread(img[i])[x][y]
        # val1 = w[i] * (c-val)
        # weighted_sum += val1
        for j in range(0,3):
            a = c[j]
            b = val[j]
            if a >= b:
                median_pixel[j] += w[i]*abs(a-b)
            else:
                median_pixel[j] += w[i]*abs(b-a)
    # print(weighted_sum)
    # weighted_sum /= sum_w
    # weighted_sum = weighted_sum.astype(int)
    # print(str(x)+'-'+str(y)+'-'+'=='+str(weighted_sum))
    # print(str(x)+' '+str(y)+' '+'['+str(median_pixel[0]/sum_w)+' '+str(median_pixel[1]/sum_w)+' '+str(median_pixel[2]/sum_w)+']')
    return (median_pixel[0]/sum_w, median_pixel[1]/sum_w, median_pixel[2]/sum_w)

In [128]:
# don't use
# to optimize - reduce 3 channels to 1
def weigtedMedianPP(img, x, y, n, w):
    median_pixel_b = []
    median_pixel_g = []
    median_pixel_r = []
    weighted_sum = np.zeros(3)
    for i in range(n):
        c = cv2.imread(img[i])[x][y]
        median_pixel_b = np.append(median_pixel_b,c[0])
        median_pixel_g = np.append(median_pixel_g,c[1])
        median_pixel_r = np.append(median_pixel_r,c[2])
    a = weighted_median(median_pixel_b, weights = w)
    b = weighted_median(median_pixel_g, weights = w)
    c = weighted_median(median_pixel_r, weights = w)
    return (a, b, c)

In [124]:
# don't use
# to optimize - reduce 3 channels to 1
def weigtedMedianPP(img, x, y, n, w, weight_sum):
    median_pixel_b = []
    median_pixel_g = []
    median_pixel_r = []
    t = 0
    for i in range(n):
        c = Image.open(img[i])
        b, g, r = c.split()
        if t == 0:
            median_pixel_b = np.append(median_pixel_b, b)
            median_pixel_g = np.append(median_pixel_g, g)
            median_pixel_r = np.append(median_pixel_r, r)
            t = 1
        else:
            median_pixel_b = np.vstack((median_pixel_b, b))
            median_pixel_g = np.vstack((median_pixel_g, g))
            median_pixel_r = np.vstack((median_pixel_r, r))            
    a = ws.numpy_weighted_median(median_pixel_b, weights = w)
    b = ws.numpy_weighted_median(median_pixel_g, weights = w)
    c = ws.numpy_weighted_median(median_pixel_r, weights = w)
    print(a)
    print(b)
    print(c)
    return (a, b, c)

In [9]:
# function to fetch and store the specific patch of respective frames for mask generation
def maskFrames(imgs, centres, N):
    new_imgs = []
    centres = centres.astype(int)
    for i in range(N):
        img = cv2.imread(imgs[i])
        v = Image.fromarray(img[(centres[i][1]-50):(centres[i][1]+50), (centres[i][0]-50):(centres[i][0]+50)])
        b, g, r = v.split()
        inter_img = Image.merge("RGB", (r, g, b))
        dirs = imgs[i].replace("target","intermediate")
        shutil.copyfile(imgs[i], dirs)
        val = "/home/shuvam/Downloads/facial-landmarks/internal/img"+str(i)+".jpg"
        inter_img.save(val)
        new_img = lip_detect(imgs[i])
        cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space/img"+str(i)+".jpg", new_img)
        new_imgs = np.append(new_imgs, val)
    return new_imgs

In [10]:
# function to calculate weighted median
def weightedMedian(S_all, S_target, target_frame, S_centres, S_centres_n, frames, target_centre, S_vec, S_vec_n, n, N, weights):
    x_num = 100
    y_num = 100
    
    weight_sum = 0.5*np.sum(weights)
    centre_x = int(target_centre[0] - (y_num/2))
    centre_y = int(target_centre[1] - (x_num/2))
    masked_images_n = maskFrames(S_vec_n, S_centres_n, n)
    im_res = np.zeros((x_num, y_num, 3))
    S_target_new = cv2.imread(target_frame)
    S_int = np.zeros((100, 100, 3))
    
    b = []
    g = []
    r = []
#     g = [[[] for x in range(x_num)] for y in range(y_num)]
#     r = [[[] for x in range(x_num)] for y in range(y_num)]
    t = 0
    b1 = []
    g1 = []
    r1 = []
    for x in range(n):
        img = Image.open(masked_images_n[x])
        B, G, R = img.split()
        B = np.asarray(B)
        G = np.asarray(G)
        R = np.asarray(R)
        if t == 0:
            b = np.append(b, np.reshape(B, (np.product(B.shape),)))
            g = np.append(g, np.reshape(G, (np.product(G.shape),)))
            r = np.append(r, np.reshape(R, (np.product(R.shape),)))
            t = 1
        else:
            b = np.vstack((b, np.reshape(B, (np.product(B.shape),))))
            g = np.vstack((g, np.reshape(G, (np.product(G.shape),))))
            r = np.vstack((r, np.reshape(R, (np.product(R.shape),))))
#                 g[i][j] = np.append(g[i][j], G)
#                 r[i][j] = np.append(r[i][j], R)
    
#     for i in range(x_num):
#         for j in range(y_num):
# #             S_target_new[int(centre_y+i)][int(centre_x+j)] = (weighted_median(median_pixel_b, weight_sum, weights = w), weighted_median(median_pixel_g, weight_sum, weights = w), weighted_median(median_pixel_r, weight_sum, weights = w))  
#             t = weighted_median(b[i][j], weight_sum, weights = w)
#             S_int[x][y] = lst
   # print(w)
    for i in range (x_num*y_num):
        # print(i)
        b1 = np.append(b1, weighted_median(b[:,i], weight_sum, weights = w))
        g1 = np.append(g1, weighted_median(g[:,i], weight_sum, weights = w))
        r1 = np.append(r1, weighted_median(r[:,i], weight_sum, weights = w))
    t = 0
    for i in range(x_num):
        for j in range(y_num):
            z = (r1[t], g1[t], b1[t])
            # S_target_new[int(centre_y+i)][int(centre_x+j)] = z
            S_int[i][j] = z
            t += 1
#     print(np.shape(b[:,1]))
    S_new = combine_frames(S_target_new, S_int, centre_x, centre_y)
    return S_new

In [11]:
# function for applying patch on target frame
def combine_frames(S_target, S_int, centre_x, centre_y):
    max = 3000
    x = 0
    y = 0
    row = np.shape(S_int)[0]
    col = np.shape(S_int)[1]
    for i in range((centre_y-10),(centre_y+10)):
        for j in range((centre_x-10),(centre_x+10)):
            val = np.linalg.norm(S_target[i][j:j+col]-S_int[0])
            if(val < max):
                max = val
                x = i
                y = j
    S_target[x:x+row, y:y+col] = S_int
    return S_target

In [44]:
# don't use
def weightedMedian(S_all, S_target, target_frame, S_centres, S_centres_n, target_centre, S_vec, S_vec_n, n, N, weights):
    x_num = 100
    y_num = 100
    # print(weights)
    weight_sum = np.sum(weights)
    centre_x = target_centre[0] - (y_num/2)
    centre_y = target_centre[1] - (x_num/2)
    masked_images_n = maskFrames(S_vec_n, S_centres_n, n)
    im_res = np.zeros((x_num, y_num, 3))
    S_target_new = cv2.imread(target_frame)
    S_int = np.zeros((100, 100, 3))
    # print(S_centres_n[0])
    # print(S_centres_n[1])
    # print(target_centre[0])
    # print(target_centre[1])
    for x in range(x_num):
        for y in range(y_num):
            lst = weigtedMedianPP(masked_images_n, x, y, n, weights)
            # S_target_new[int(centre_y+x)][int(centre_x+y)] = lst
            S_int[x][y] = lst
    return S_target_new, S_int.astype(int)

In [27]:
# used for generating PCA from video
def find_PCA(video_link, folder):
    vs = FileVideoStream(video_link).start()
    time.sleep(2.0)
    frameNo = 1
    t = 0
    imgs_rslt = []
    centres = []
    frame_list = []
    length = int(cv2.VideoCapture(video_link).get(cv2.CAP_PROP_FRAME_COUNT))

    # loop over the frames from the video stream
    while True:
        # grab the frame from the threaded video stream, resize it to
        # have a maximum width of 600 pixels, and convert it to
        # grayscale
        
        frame = vs.read()
        # frame = np.rot90(frame)
        
        frame = imutils.resize(frame, width=600)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces in the grayscale frame
        rects = detector(gray, 0)
        
        # loop over the face detections
        for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            faceAligned = fa.align(frame, gray, rect)
            dirs = folder + str(999+frameNo) + ".jpg"
            cv2.imwrite(dirs, faceAligned)
            rectan = dlib.rectangle(0, 0, 256, 256)
            grays = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)

            shape = predictor(grays, rectan)
            shape = face_utils.shape_to_np(shape)
            
            # reduced shapes
#             shapes = shape[60:68]
            
            
            # changed implementation
            shapes = shape[48:68]
            angle = calcAngle(shapes, frameNo)
            
            # original implementation
#             shapes = np.concatenate([shape[48:61], shape[62:65]])
#             shapes = np.concatenate([shapes, shape[66:69]])
            
            
            imgs = np.reshape(shapes, (np.product(shapes.shape),))
            
            if t == 0:
                centres = np.append(centres, np.sum(shapes, axis=0))
                imgs_rslt = np.append(imgs_rslt,imgs)
                t = 1

            else:
                centres = np.vstack((centres, np.sum(shapes, axis=0)))
                imgs_rslt = np.vstack((imgs_rslt,imgs))
            
            frame_list = np.append(frame_list, dirs)

        # increment frame count
        if frameNo < length:
            frameNo += 1
        else:
            vs.stop()
            break

    imgs_rslt = imgs_rslt.astype(int)
    centres = (centres/[20, 20]).astype(int)
    # print(centres)
    pca = PCA(n_components=20)
    img_pca = pca.fit_transform(imgs_rslt)
    print("Done here...")
    return imgs_rslt, img_pca, frame_list, length, centres

In [31]:
# modified image loading
# generates PCA from saved images
def find_PCA1(folder):
    time.sleep(2.0)
    frameNo = 1000
    t = 0
    imgs_rslt = []
    centres = []
    angles = []
    frame_list = []
    imagesList = sorted(listdir(folder))
    length = len(imagesList)

    # loop over the frames from the video stream
    for image in imagesList:
        # grab the frame from the threaded video stream, resize it to
        # have a maximum width of 600 pixels, and convert it to
        # grayscale
        
        frame = cv2.imread(folder + image)
        
        if frameNo >= 1000:
            frame = imutils.resize(frame, width=600)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # detect faces in the grayscale frame
            rects = detector(gray, 0)

            # loop over the face detections
            for rect in rects:
                # determine the facial landmarks for the face region, then
                # convert the facial landmark (x, y)-coordinates to a NumPy
                # array
                faceAligned = fa.align(frame, gray, rect)
                dirs = folder + image
                rectan = dlib.rectangle(0, 0, 256, 256)
                grays = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)

                shape = predictor(grays, rectan)
                shape = face_utils.shape_to_np(shape)
                #print('for frame'+str(frameNo))
            # reduced shapes
                shapes = shape[48:68]
                angle = calcAngle(shapes, frameNo)
            
            # changed implementation
#                shapes = shape[48:68]
            
            # original implementation
#             shapes = np.concatenate([shape[48:61], shape[62:65]])
#             shapes = np.concatenate([shapes, shape[66:69]])
            
            
                imgs = np.reshape(shapes, (np.product(shapes.shape),))

                if t == 0:
                    centres = np.append(centres, np.sum(shapes, axis=0))
                    imgs_rslt = np.append(imgs_rslt,imgs)
                    angles = np.append(angles,angle)
                    t = 1

                else:
                    centres = np.vstack((centres, np.sum(shapes, axis=0)))
                    imgs_rslt = np.vstack((imgs_rslt,imgs))
                    angles = np.vstack((angles,angle))

                frame_list = np.append(frame_list, dirs)

        # increment frame count
        frameNo += 1
        if frameNo == 2500:
            break

    imgs_rslt = imgs_rslt.astype(int)
    centres = (centres/[20, 20]).astype(int)
    # print(centres)
    pca = PCA(n_components=16)
    img_pca = pca.fit_transform(imgs_rslt)
    print("Done here...")
    return imgs_rslt, img_pca, frame_list, length, centres, angles

In [65]:
# test generated patch
# don't use
def find_PCA(video_link, folder):
    vs = FileVideoStream(video_link).start()
    time.sleep(2.0)
    frameNo = 1
    t = 0
    imgs_rslt = []
    centres = []
    frame_list = []
    length = int(cv2.VideoCapture(video_link).get(cv2.CAP_PROP_FRAME_COUNT))

    # loop over the frames from the video stream
    while True:
        # grab the frame from the threaded video stream, resize it to
        # have a maximum width of 600 pixels, and convert it to
        # grayscale
        frame = vs.read()
        frame = imutils.resize(frame, width=600)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces in the grayscale frame
        rects = detector(gray, 0)
        
        # loop over the face detections
        for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            # faceAligned = fa.align(frame, gray, rect)
            
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            print(x)
            print(x+w)
            print(y)
            print(h+y)
            print('')
            frame = frame[y:y+h, x:x+w]
            print(np.shape(frame))
            
            dirs = folder + str(frameNo) + ".jpg"
            cv2.imwrite(dirs, frame)
            
            
            rectan = dlib.rectangle(0, 0, 256, 256)
            grays = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            shape = predictor(grays, rect)
            shape = face_utils.shape_to_np(shape)
            
            # reduced shapes
#             shapes = shape[60:68]
            
            
            # changed implementation
            shapes = shape[48:68]
            
            # original implementation
#             shapes = np.concatenate([shape[48:61], shape[62:65]])
#             shapes = np.concatenate([shapes, shape[66:69]])
            
            
            imgs = np.reshape(shapes, (np.product(shapes.shape),))
            
            if t == 0:
                centres = np.append(centres, np.sum(shapes, axis=0))
                imgs_rslt = np.append(imgs_rslt,imgs)
                t = 1

            else:
                centres = np.vstack((centres, np.sum(shapes, axis=0)))
                imgs_rslt = np.vstack((imgs_rslt,imgs))

        # increment frame count
        frame_list = np.append(frame_list, dirs)
        if frameNo < length:
            frameNo += 1
        else:
            vs.stop()
            break

    imgs_rslt = imgs_rslt.astype(int)
    centres = (centres/[20, 20]).astype(int)
    # print(centres)
    pca = PCA(n_components=20)
    img_pca = pca.fit_transform(imgs_rslt)
    return imgs_rslt, img_pca, frame_list, length, centres

In [13]:
# proto func to calc similarity based on angles between landmarks
def calcAngle(shapes, frameNo):
    refPoint1 = shapes[0]
    refPoint2 = shapes[6]
    shape = np.concatenate([shapes[1:6], shapes[7:]])
    result = []
    line1 = refPoint1 - refPoint2
    j = 0
    for i in shape:
        line2 = refPoint1 - i
        cosine_angle = np.dot(line1, line2) / (np.linalg.norm(line1) * np.linalg.norm(line2))
        angle = np.arccos(cosine_angle)
        # print(str(frameNo)+' '+str(j)+' '+str(angle)+' '+str(cosine_angle)+' '+str(line1)+' '+str(line2))
        result = np.append(result, np.degrees(angle))
        j += 1
    return result

In [14]:
# func to calc l2 similarity 
def similarity(src_pca, tgt_pca, len):
    dist = []
    # print(len)
    for i in range(len):
        if i == 0:
            #print("here")
            dist = np.append(dist, [i+1, np.linalg.norm(src_pca[i] - tgt_pca)])
        else:
            dist = np.vstack((dist, [i+1, np.linalg.norm(src_pca[i] - tgt_pca)]))
    #print(dist)
    dist = dist[dist[:,1].argsort()]
    return dist

In [15]:
# pre-processing

#sources_vec, sources_pca, sources_frame, sources_len, sources_centre, sources_angles = find_PCA("aud.mp4","/home/shuvam/Downloads/facial-landmarks/source/")
source_vec, source_pca, source_frame, source_len, source_centre, source_angles = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source/")
#source_vec, source_pca, source_frame, source_len, source_centre, source_angles = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source/")
target_vec, target_pca, target_frame, target_len, target_centre, target_angles = find_PCA1("/home/shuvam/Downloads/facial-landmarks/target/")


Done here...
Done here...


In [89]:
# generate frames
n = 30
target_len = len(target_vec)
source_len = len(source_vec)
for i in range(0,1):
    start_time = time.time()
    y = []
    centres = []
    rslt = similarity(target_angles, source_angles[i], target_len)
    frames = []
    rslt1 = rslt
    rslt = rslt[:n]
    t = 0
    # print(source_frame)
    new_imgn = lip_detect(source_frame[i])
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space/src_img.jpg", new_imgn)
    for z in rslt[:,0].astype(int):
        if t == 0:
            y = np.append(y, target_vec[z-1])
            centres = np.append(centres, target_centre[z-1])
            frames = np.append(frames, target_frame[z-1])
            t = 1
        else:
            y = np.vstack((y, target_vec[z-1]))
            centres = np.vstack((centres, target_centre[z-1]))
            frames = np.append(frames, target_frame[z-1])
    y = y.astype(int)
    print("Working on frame "+str(i))
    sigma = calcSigma(y, target_vec, source_vec[i], n, target_len)
    # sigma = 15
    print(sigma)
    weight = calcWeight(y, source_vec[i], n, sigma)
    # print(weight)
    gen_img = weightedMedian(source_vec, target_vec[i], target_frame[i], source_centre, centres, frames, target_centre[i], source_frame, frames, n, target_len, weight)
    gen_img = Image.fromarray(gen_img)
    b, g, r = gen_img.split()
    gen_img = Image.merge("RGB", (r, g, b))
    gen_img.save("/home/shuvam/Downloads/facial-landmarks/inter/img"+('0'*(len(str(target_len))-len(str(i))))+str(i)+".jpg")
    elapsed_time = time.time() - start_time
    print("Done")
    print("Elapsed time - " + str(elapsed_time)+'\n')

Working on frame 0
4
Done
Elapsed time - 4.289734363555908



In [55]:
# test - don't use

#source_vec1, source_pca1, source_frame1, source_len1, source_centre1, source_angles1 = find_PCA("cut123.mp4","/home/shuvam/Downloads/facial-landmarks/source1/")
#source_vec2, source_pca2, source_frame2, source_len2, source_centre2, source_angles2 = find_PCA("fun123.mp4","/home/shuvam/Downloads/facial-landmarks/source2/")
#source_vec3, source_pca3, source_frame3, source_len3, source_centre3, source_angles3 = find_PCA("output123.mp4","/home/shuvam/Downloads/facial-landmarks/source3/")
# source_vec1, source_pca1, source_frame1, source_len1, source_centre1, source_angles1 = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source1/")
# source_vec2, source_pca2, source_frame2, source_len2, source_centre2, source_angles2 = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source2/")
# source_vec3, source_pca3, source_frame3, source_len3, source_centre3, source_angles3 = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source3/")
for i in range(source_len1):
    new_imgn, new_imgx = lip_detect(source_frame1[i])
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space(cut123)/src_img"+str(1000+i)+".jpg", new_imgn)
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space_x(cut123)/src_img"+str(1000+i)+".jpg", new_imgx)
for i in range(source_len2):
    new_imgn, new_imgx = lip_detect(source_frame2[i])
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space(fun123)/src_img"+str(1000+i)+".jpg", new_imgn)
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space_x(fun123)/src_img"+str(1000+i)+".jpg", new_imgx)
for i in range(source_len3):
    new_imgn, new_imgx = lip_detect(source_frame3[i])
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space(output123)/src_img"+str(1000+i)+".jpg", new_imgn)
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space_x(output123)/src_img"+str(1000+i)+".jpg", new_imgx)

In [32]:
# test - don't use

#source_vec, source_pca, source_frame, source_len, source_centre, source_angles = find_PCA("cut123.mp4","/home/shuvam/Downloads/facial-landmarks/source/")
#source_vec, source_pca, source_frame, source_len, source_centre, source_angles = find_PCA1("/home/shuvam/Downloads/facial-landmarks/source/")
for i in range(source_len):
    new_imgn, new_imgx = lip_detect(source_frame[i])
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space/src_img"+str(1000+i)+".jpg", new_imgn)
    cv2.imwrite("/home/shuvam/Downloads/facial-landmarks/space_x/src_img"+str(1000+i)+".jpg", new_imgx)

Done here...


In [168]:
# test - don't use

print(np.shape(rslt1))
print(target_len)
print(source_frame[75])
print(similarity(target_angles, source_angles[75], target_len))
print(similarity(target_angles[966:969], source_angles[75], 2))
# print(target_frame[0:5])
# print(frames[0:5])

(1500, 2)
1500
/home/shuvam/Downloads/facial-landmarks/source/1075.jpg
[[  9.67000000e+02   5.10171821e-01]
 [  2.90000000e+01   7.88856367e-01]
 [  9.16000000e+02   9.77034278e-01]
 ..., 
 [  1.96000000e+02              nan]
 [  2.25000000e+02              nan]
 [  2.26000000e+02              nan]]
[[  1.           0.51017182]
 [  2.          38.29837635]]


In [112]:
# test - don't use

n = 50
target_len = len(target_vec)
y = []
rslt = similarity(target_vec, source_vec[108], target_len)
rslt = rslt[:n]
print(rslt)
t = 0
for z in rslt[:,0].astype(int):
    if t == 0:
        y = np.append(y, target_vec[z-1])
        t = 1
    else:
        y = np.vstack((y, target_vec[z-1]))
y = y.astype(int)
sigma = calcSigma(y, target_vec, source_vec[107], n, target_len)
print(sigma)

[[ 4473.            10.95445115]
 [ 3727.            13.07669683]
 [ 4409.            13.22875656]
 [ 4408.            14.03566885]
 [ 3720.            15.03329638]
 [ 3700.            15.06651917]
 [ 3726.            15.13274595]
 [ 3823.            15.13274595]
 [ 3830.            15.39480432]
 [ 3725.            15.49193338]
 [ 2703.            15.71623365]
 [ 4472.            15.8113883 ]
 [ 3701.            15.87450787]
 [ 3721.            16.0623784 ]
 [ 3931.            16.2788206 ]
 [ 3744.            16.2788206 ]
 [ 2701.            16.55294536]
 [ 3719.            16.64331698]
 [ 3724.            16.70329309]
 [ 2704.            16.73320053]
 [ 2700.            16.73320053]
 [ 3557.            16.76305461]
 [ 3738.            16.79285562]
 [ 4410.            16.82260384]
 [ 2702.            16.85229955]
 [ 3745.            16.91153453]
 [ 4412.            17.        ]
 [ 3829.            17.11724277]
 [ 4414.            17.20465053]
 [  415.            17.29161647]
 [ 3558.  

In [71]:
# test - don't use

# Image.fromarray(int_img1.astype(np.uint8)).show()
print(np.shape(target_pca-source_pca[i]))
print(np.sqrt(np.sum((target_pca-source_pca[i])**2, axis = 1)))
print(rslt1)
for i in range(10):
    lt = Image.fromarray(cv2.imread(frames[i]))
    b,g,r = lt.split()
    lt = Image.merge("RGB", (r, g, b))
    lt.save("/home/shuvam/Downloads/facial-landmarks/rsltset/img"+str(i)+".jpg")
    lt.show()

(150, 25)
[  27.36274246   27.36274246   27.36274246   27.36274246   27.36274246
   27.36274246   27.36274246   27.36274246   27.36274246   27.36274246
   27.36274246   27.36274246   27.36274246   27.36274246   27.36274246
   27.36274246   27.36274246   27.36274246   27.36274246   27.36274246
   27.36274246   27.36274246   27.36274246   27.36274246   27.36274246
   27.36274246   56.65156229   87.33453298   87.33453298   87.33453298
   87.33453298   87.33453298   56.65156229   56.65156229   56.65156229
   56.65156229   56.65156229   56.65156229   27.36274246   27.36274246
   27.36274246   56.65156229   27.36274246   39.41564883   39.41564883
   59.42275555   59.42275555   59.42275555   59.42275555   39.41564883
   39.41564883   39.41564883   59.42275555   59.42275555   59.42275555
   59.42275555   59.42275555   59.42275555  106.75652494   59.42275555
   59.42275555   59.42275555   59.42275555  106.75652494  106.75652494
  106.75652494  106.75652494  106.75652494  106.75652494  106.75652

In [15]:
# function to combine frames to generate video
# incomplete

img_names = "img%0"+(str(target_len)+".jpg"
FFMPEG_BIN = "ffmpeg"
command = [ FFMPEG_BIN,
        '-y', # (optional) overwrite output file if it exists
        '-f', 'rawvideo',
        '-vcodec','rawvideo',
        '-s', '256x256', # size of one frame
        '-pix_fmt', 'rgb24',
        '-r', '30', # frames per second
        '-i', img_names, # The imput comes from a pipe
        '-i', '/home/shuvam/Downloads/facial-landmarks/target/cut2.mp4',
        '-map', '1:a:0',
        '-shortest',
        '-vcodec', 'mpeg'",
        'result.mp4' ]

pipe = sp.Popen( command, stdin=sp.PIPE, stderr=sp.PIPE)

SyntaxError: invalid syntax (<ipython-input-15-5f6e61e9be66>, line 2)